<a href="https://colab.research.google.com/github/hss0727/BigData_project_Spark/blob/main/TopicModeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
!pip install findspark
!pip install pyspark
!pip install sparknlp
!pip install nltk


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 51 kB/s 
     |████████████████████████████████| 199 kB 85.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=e75d1ccaa2265f9fe23c9cb916d6f8e6a56b1fb06bfc88d5ff864df3e40a3b52
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 648 kB 5.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from sparknlp.pretrained import PretrainedPipeline
import sparknlp
import pyspark.sql.functions as F
from pyspark.sql.functions import col

In [5]:
# Start Spark Session with Spark NLP
spark = sparknlp.start()

In [7]:
submissions_en = spark.read.parquet("/content/drive/Shareddrives/502_project/data/submissions_en_cleaned")

In [8]:
submissions_en.show(3)

+------------------+------------------+--------+-----------------+--------------+----------------------+-----------------+---------+----------+------------+-------------------+----------------+---------------------+----------------+-------------+----------------+---------------+------+----------+---------+------+------+----------+--------+------+---------+----------------+----------------------+-------+--------+--------------------+--------------------+------+-----+--------------------+-------------+------------+--------------+-------------+-------+-----------------------+--------------------+------+---------+-------+--------+-----------+---------------------+------------+------------+-----+------------+--------------------+---------+-------+--------+-----------------+------------+--------------+--------------------+--------------------+----------------------+--------------------+----------------+---------------+--------------------+--------------------+----------------+------------+--

In [27]:
text = submissions_en.select('cleaned_text')

In [28]:
"""
Reference: Topic Modeling with Pyspark
https://medium.com/trustyou-engineering/topic-modelling-with-pyspark-and-spark-nlp-a99d063f1a6e
"""

'\nReference: Topic Modeling with Pyspark\nhttps://medium.com/trustyou-engineering/topic-modelling-with-pyspark-and-spark-nlp-a99d063f1a6e\n'

In [29]:
# Transform data to annotation format using DocumentAssembler
from sparknlp.base import DocumentAssembler
documentAssembler = DocumentAssembler() \
  .setInputCol('cleaned_text')  \
  .setOutputCol('document')

In [30]:
# Setup Tokenizer
from sparknlp.annotator import Tokenizer
tokenizer = Tokenizer() \
  .setInputCols(['document']) \
  .setOutputCol('tokenized')

In [31]:
# Setup Normalization
from sparknlp.annotator import Normalizer
normalizer = Normalizer() \
  .setInputCols(['tokenized'])  \
  .setOutputCol('normalized') \
  .setLowercase(True)

In [32]:
# Setup Lemmatizer
from sparknlp.annotator import LemmatizerModel
lemmatizer = LemmatizerModel.pretrained() \
     .setInputCols(['normalized']) \
     .setOutputCol('lemmatized')

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


In [33]:
# Setup stopwords
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
eng_stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
type(eng_stopwords)

list

In [35]:
# Add stopwords
new_stop_words = ['russia','russian','ukraine','ukrainian','delete']
eng_stopwords = eng_stopwords + new_stop_words

In [36]:
# Setup stop words cleaner
from sparknlp.annotator import StopWordsCleaner
stopwords_cleaner = StopWordsCleaner() \
     .setInputCols(['lemmatized']) \
     .setOutputCol('no_stop_lemmatized') \
     .setStopWords(eng_stopwords)

In [37]:
# Transformation from the annotation format of SparkNLP to a human-readable format
from sparknlp.base import Finisher
finisher = Finisher() \
     .setInputCols(['no_stop_lemmatized']) \
     .setIncludeMetadata(True)

In [38]:
# Define pipeline
from pyspark.ml import Pipeline
pipeline = Pipeline() \
     .setStages([documentAssembler,
                 tokenizer,
                 normalizer,
                 lemmatizer,
                 stopwords_cleaner,
                 finisher])

In [39]:
processed_text = pipeline.fit(text).transform(text)

In [40]:
processed_text.show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|cleaned_text                                                                                                                                                                              |finished_no_stop_lemmatized                                                                  

In [41]:
from pyspark.ml.feature import CountVectorizer
tfizer = CountVectorizer(inputCol='finished_no_stop_lemmatized',
                         outputCol='tf_features')
tf_model = tfizer.fit(processed_text)
tf_result = tf_model.transform(processed_text)

In [42]:
tf_result.show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|cleaned_text                                                                                   

In [43]:
from pyspark.ml.feature import IDF
idfizer = IDF(inputCol='tf_features', 
              outputCol='tf_idf_features')
idf_model = idfizer.fit(tf_result)
tfidf_result = idf_model.transform(tf_result)

In [44]:
tfidf_result.select('tf_idf_features').show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|tf_idf_features                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
+-----

In [45]:
from pyspark.ml.clustering import LDA
num_topics = 10
max_iter = 10
lda = LDA(k=num_topics, 
          maxIter=max_iter, 
          featuresCol='tf_idf_features')
lda_model = lda.fit(tfidf_result)

In [46]:
import pyspark.sql.types as T

vocab = tf_model.vocabulary

def get_words(token_list):
    return [vocab[token_id] for token_id in token_list]

udf_to_words = F.udf(get_words, T.ArrayType(T.StringType()))

In [47]:
num_top_words = 10

topics = lda_model  \
     .describeTopics(num_top_words) \
     .withColumn('topicWords', udf_to_words(F.col('termIndices')))

topics.select('topic', 'topicWords').show(truncate=100)

+-----+------------------------------------------------------------------------------------+
|topic|                                                                          topicWords|
+-----+------------------------------------------------------------------------------------+
|    0|                          [soldier, amp, call, war, f, phone, https, border, b, get]|
|    1|           [island, snake, moskva, dog, sink, pow, shoot, soldier, twitter, torture]|
|    2|    [azov, weapon, germany, mariupol, war, azovstal, president, putin, say, soldier]|
|    3|               [missile, fly, air, bridge, help, plant, defense, nuclear, su, force]|
|    4|[vehicle, destroy, tank, force, armored, drone, artillery, troop, equipment, turkey]|
|    5|             [live, mariupol, news, radio, troll, war, twitter, steel, map, theater]|
|    6| [gt, nazi, denazification, putin, negotiation, nazism, war, state, airstrike, must]|
|    7|                         [eu, gas, oil, say, putin, war, city, 